In [2]:
!pip install -q -U google-genai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.2/261.2 kB 7.1 MB/s eta 0:00:00:00:01


In [36]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")

import os
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

USE_FAKE_LLM = False  # Set True if key disabled

from google import genai


In [37]:
import json
import uuid
import datetime
import logging
from dataclasses import dataclass, asdict, field
from typing import List, Dict, Any, Optional

logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s] [%(levelname)s] %(name)s - %(message)s",
)
logger = logging.getLogger("study_planner")


In [38]:
class LLMClient:
    def __init__(self, model_name="models/gemini-flash-latest"):
        self.model_name = model_name
        api_key = os.environ.get("GOOGLE_API_KEY")

        if not USE_FAKE_LLM and api_key:
            self.client = genai.Client(api_key=api_key)
            logger.info("Using REAL Gemini model: %s", model_name)
        else:
            self.client = None
            logger.info("Using FAKE mode (offline safe).")

    def generate(self, prompt: str) -> str:
        if self.client is None:
            return (
                "FAKE LLM RESPONSE (offline safe)\n\n"
                f"Prompt snippet:\n{prompt[:300]} ..."
            )
        try:
            response = self.client.models.generate_content(
                model=self.model_name,
                contents=[{"role":"user","parts":[{"text":prompt}]}],
            )
            return response.text
        except Exception as e:
            logger.error("Gemini error: %s", e)
            return f"LLM ERROR: {e}"


In [39]:
@dataclass
class SubjectConfig:
    name: str
    difficulty: str
    exam_date: Optional[str] = None

@dataclass
class UserInput:
    user_id: str
    total_days: int
    daily_study_hours: float
    subjects: List[SubjectConfig]
    weak_areas: List[str] = field(default_factory=list)
    goals: str = ""

@dataclass
class StudyPlan:
    user_id: str
    created_at: str
    schedule_text: str
    resources_text: str
    evaluation_text: str = ""
    metadata: Dict[str, Any] = field(default_factory=dict)

@dataclass
class ProgressUpdate:
    date: str
    completed_hours: float
    notes: str = ""
    pending_topics: List[str] = field(default_factory=list)


In [40]:
class MemoryManager:
    def __init__(self, path="study_memory.json"):
        self.path = path
        self._data = self._load()

    def _load(self):
        try:
            with open(self.path, "r") as f:
                return json.load(f)
        except:
            return {}

    def _save(self):
        with open(self.path, "w") as f:
            json.dump(self._data, f, indent=2)

    def get_user(self, user_id):
        return self._data.setdefault(user_id, {"plans": [], "progress": []})

    def add_plan(self, plan: StudyPlan):
        self.get_user(plan.user_id)["plans"].append(asdict(plan))
        self._save()

    def add_progress(self, user_id: str, progress: ProgressUpdate):
        self.get_user(user_id)["progress"].append(asdict(progress))
        self._save()


In [41]:
class BaseAgent:
    def __init__(self, name, llm=None, memory=None):
        self.name = name
        self.llm = llm
        self.memory = memory
        self.logger = logging.getLogger(f"Agent.{name}")

def study_tips_tool(subject):
    return (
        f"Study tips for {subject}:\n"
        "- Revise previous session first\n"
        "- Use active recall + spaced repetition\n"
        "- Practice 5–10 previous questions\n"
    )


In [42]:
class InputAgent(BaseAgent):
    def collect(self, subjects, total_days, daily_hours, weak_areas, goals, user_id=None):
        uid = user_id or str(uuid.uuid4())
        subject_objs = [SubjectConfig(**s) for s in subjects]
        return UserInput(uid, total_days, daily_hours, subject_objs, weak_areas, goals)

class PlanningAgent(BaseAgent):
    def create_plan(self, user_input):
        prompt = f"""
Create a {user_input.total_days}-day study schedule
for {user_input.daily_study_hours} hrs/day.
Subjects: {[s.name for s in user_input.subjects]}
Weak areas: {user_input.weak_areas}
Goals: {user_input.goals}
"""
        text = self.llm.generate(prompt)
        return StudyPlan(user_input.user_id, datetime.datetime.utcnow().isoformat(), text, "")

class ResourceSearchAgent(BaseAgent):
    def add_resources(self, user_input, plan):
        tips = "\n\n".join(study_tips_tool(s.name) for s in user_input.subjects)
        query_prompt = f"Suggest YouTube queries for {', '.join(s.name for s in user_input.subjects)}"
        plan.resources_text = plan.resources_text + "\n\n" + tips + "\n\n" + query_prompt
        return plan

class EvaluationAgent(BaseAgent):
    def evaluate(self, user_input, plan):
        prompt = f"Evaluate study plan and rate from 1-10:\n\n{plan.schedule_text}"
        plan.evaluation_text = self.llm.generate(prompt)
        return plan

class ProgressTrackerAgent(BaseAgent):
    def record(self, user_id, hours, notes=""):
        p = ProgressUpdate(datetime.date.today().isoformat(), hours, notes)
        self.memory.add_progress(user_id, p)
        return f"Logged {hours} hours"

class ReminderAgent(BaseAgent):
    def remind(self, goal):
        return f"Reminder: stay consistent & remember your goal → {goal}"


In [43]:
class StudyPlannerOrchestrator:
    def __init__(self):
        self.memory = MemoryManager()
        self.llm = LLMClient()

        self.input = InputAgent("Input", memory=self.memory)
        self.plan = PlanningAgent("Planner", llm=self.llm, memory=self.memory)
        self.resource = ResourceSearchAgent("Resources", llm=self.llm, memory=self.memory)
        self.eval = EvaluationAgent("Eval", llm=self.llm, memory=self.memory)
        self.progress = ProgressTrackerAgent("Progress", llm=self.llm, memory=self.memory)
        self.reminder = ReminderAgent("Reminder", llm=self.llm, memory=self.memory)

    def create_plan(self, **kwargs):
        user_input = self.input.collect(**kwargs)
        plan = self.plan.create_plan(user_input)
        plan = self.resource.add_resources(user_input, plan)
        plan = self.eval.evaluate(user_input, plan)
        self.memory.add_plan(plan)
        return plan

    def log(self, user_id, hours, notes=""):
        return self.progress.record(user_id, hours, notes)

    def remind(self, plan):
        return self.reminder.remind(plan.metadata.get("goal", "Target Score"))


In [44]:
orch = StudyPlannerOrchestrator()

subjects = [
    {"name": "Math", "difficulty": "hard", "exam_date": "2025-12-15"},
    {"name": "Physics", "difficulty": "medium", "exam_date": "2025-12-17"},
    {"name": "Chemistry", "difficulty": "hard", "exam_date": "2025-12-20"},
]

plan = orch.create_plan(
    subjects=subjects,
    total_days=15,
    daily_hours=5.0,
    weak_areas=["Organic chemistry", "Calculus"],
    goals="Score 85% or more"
)

print(plan.schedule_text)
print("\nRESOURCES:\n", plan.resources_text)
print("\nEVALUATION:\n", plan.evaluation_text)

print("\nPROGRESS:", orch.log(plan.user_id, 4.0, "Completed Math revision"))
print("\nREMINDER:", orch.remind(plan))


[2025-11-29 15:09:38,728] [INFO] study_planner - Using REAL Gemini model: models/gemini-flash-latest
[2025-11-29 15:09:38,734] [INFO] google_genai.models - AFC is enabled with max remote calls: 10.
[2025-11-29 15:09:50,205] [INFO] httpx - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-flash-latest:generateContent "HTTP/1.1 200 OK"
[2025-11-29 15:09:50,209] [INFO] google_genai.models - AFC is enabled with max remote calls: 10.
[2025-11-29 15:10:03,785] [INFO] httpx - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-flash-latest:generateContent "HTTP/1.1 200 OK"


This 15-day study schedule is designed to maximize your efficiency within the 5.0 hours per day limit, focusing strategically on your weak areas (Organic Chemistry and Calculus) while maintaining proficiency in the other core subjects.

The schedule incorporates a mix of active learning, spaced repetition (review), and targeted practice to help you achieve your goal of scoring **85% or more**.

---

## 15-DAY HIGH-INTENSITY STUDY SCHEDULE

**Constraints:** 5.0 hours/day | **Subjects:** Math, Physics, Chemistry
**Focus Areas:** Organic Chemistry, Calculus
**Goal:** Score 85%+

### Key Principles

1.  **Morning Focus (Weak Areas):** The first study block (usually the most productive) is dedicated to your weak areas (Calculus and Organic Chemistry).
2.  **Rotation:** Subjects are rotated daily to prevent burnout and ensure consistent exposure.
3.  **Active Recall:** Sessions include time for practice problems, which is essential for concept retention.
4.  **Daily Review:** A short 30-minu